In [145]:
import numpy as np
import matplotlib.pyplot as plt
import os
from enum import Enum
from itertools import permutations
from copy import copy
from icecream import ic

### Input format
The first line is of the form
```
agent_count est_o dl_o
```

It is followed by `agent_count` lines, of which line $i$ (zero-indexed) of the input .csv file pertains to issue $i$, and is a tuple of the form `(agent, parent_issue, [pretasks], children_finish, issue_type, other_issue)`, where

- `agent` is the agent ID of that agent which is supposed to do this task
- `parent_issue` of an incoming issue is **not** the same as that of its `other_issue`
- `[pretasks]` = Set of explicit pretasks (i.e., doesn't include parent-child relationships). '' if there is no pretask. Separated by ';' otherwise
- `children_finish` = 0 for "and", 1 for "or", '' for no chlidren. It may be 0 or 1 even if it has no children; however, '' indicates that we know for sure that there is no child.
- `issue_type` = 1 for local tasks (that are not incoming), 2 for outgoing tasks, 3 for incoming tasks
- `other_issue` is the issue ID of the issue linked to this issue (defined only when issue_type is outgoing or incoming)


In [146]:
class issue_type(Enum):
    NONE = 0
    LOCAL = 1
    OUTGOING = 2
    INCOMING = 3


class issue:
    def __init__(self, issue_num: int, line: str) -> None:
        self.ID = issue_num
        line = line.split(',')
        self.agent = int(line[0])
        self.parent = int(line[1])
        try:
            self.pretasks = list(map(int, line[2].split(';')))
        except:
            self.pretasks = []

        self.children_finish = int(line[3])
        self.issue_type = issue_type(int(line[4]))
        self.other_issue = int(line[5])


class agent:
    def __init__(self) -> None:
        self.issues: list[int] = []
        # issues that are either outgoing or incoming
        negotiation_issues: list[int] = []


In [147]:
# input_file = ".." + os.sep + "data" + os.sep + "input.txt"
input_file = os.path.join(os.getcwd(), '..', 'data', 'input.csv')

issue_graph: list[issue] = []
with open(input_file, 'r') as f:
    agent_count, est_o, dl_o = list(map(int, f.readline().split()))
    agents: list[agent] = [agent() for i in range(agent_count)]
    issue_num: int = 0
    for line in f:
        cur_issue = issue(issue_num, line)
        issue_graph.append(cur_issue)
        agents[cur_issue.agent].issues.append(issue_num)
        issue_num += 1

ic(est_o)
ic(dl_o)
ic(agent_count)
print("Agents:")
for i, agent in enumerate(agents):
    print(f"Agent {i}: {agent.issues}")
print("\nIssues:")
for issue in issue_graph:
    print(issue.agent, issue.parent, issue.pretasks, issue.children_finish, issue.issue_type.name, issue.other_issue)

ic| est_o: 0
ic| dl_o: 100
ic| agent_count: 3


Agents:
Agent 0: [0, 1]
Agent 1: [2, 3, 4, 5, 6, 7, 8]
Agent 2: [9, 10, 11, 12]

Issues:
0 -1 [] 0 OUTGOING 2
0 -1 [] 0 OUTGOING 10
1 -1 [] 0 INCOMING 0
1 2 [] 0 LOCAL -1
1 3 [] 0 LOCAL -1
1 3 [4] 0 LOCAL -1
1 4 [] 0 OUTGOING 9
1 4 [] 0 LOCAL -1
1 4 [6, 7] 0 LOCAL -1
2 -1 [] 0 INCOMING 6
2 -1 [] 0 INCOMING 1
2 9 [] 0 LOCAL -1
2 10 [] 0 LOCAL -1


Finding those issues that are going to be negotiated over:

In [148]:
print("Negotiation issues:")
for i, agent in enumerate(agents):
    agent.negotiation_issues = [issueID for issueID in agent.issues if issue_graph[issueID].issue_type in [
        issue_type.OUTGOING, issue_type.INCOMING]]
    agent.negotiation_issues.sort()
    print(f"Agent {i}: {agent.negotiation_issues}")


Negotiation issues:
Agent 0: [0, 1]
Agent 1: [2, 6]
Agent 2: [9, 10]


Complete search for the optimal (negotiation ordering, feature assignment tuple)

In [149]:
for agent in agents:
    best_value = float('-inf')
    best_ordering = None
    best_assignment = None
    for ordering in permutations(agent.negotiation_issues):
        # TODO: GENERATE ALL POSSIBLE ASSIGNMENTS
        # for assignment in 
        pass